In [1]:
#Directory Structure
#~/Documents/TAQData <--- has the parquet files
#~/gitcode/AmitG/TAQCode <--- has the jupypter and python code


In [28]:
import pyarrow.parquet as pq
from datetime import datetime
# ---- quotes file ----
quotes_path = '~/Documents/TAQData/data_quotes_2023_12_15.parquet'
pf_quotes = pq.ParquetFile(quotes_path)
print("🧾 Quotes schema:")
print(pf_quotes.schema_arrow)     # column names and types only

# ---- trades file ----
trades_path = '~/Documents/TAQData/data_trades_2023_12_15.parquet'
pf_trades = pq.ParquetFile(trades_path)
print("\n💹 Trades schema:")
print(pf_trades.schema_arrow)


🧾 Quotes schema:
DATE: date32[day]
TIME_M: time64[ns]
EX: large_string
BID: double
BIDSIZ: int64
ASK: double
ASKSIZ: int64
QU_COND: large_string
QU_SEQNUM: int64
NATBBO_IND: large_string
QU_CANCEL: large_string
QU_SOURCE: large_string
SYM_ROOT: large_string
SYM_SUFFIX: large_string

💹 Trades schema:
date: date32[day]
time_m: time64[us]
time_m_nano: int16
ex: string
sym_root: string
sym_suffix: string
tr_scond: string
size: int32
price: double
tr_stop_ind: string
tr_corr: string
tr_seqnum: int64
tr_id: string
tr_source: string
tr_rf: string
part_time: time64[us]
part_time_nano: int16
trf_time: time64[us]
trf_time_nano: int16
tte_ind: string


In [5]:
import pyarrow.parquet as pq

# Paths to your files
quotes_path = '~/Documents/TAQData/data_quotes_2023_12_15.parquet'
trades_path = '~/Documents/TAQData/data_trades_2023_12_15.parquet'

# Efficiently count rows without loading data
quotes_rows = pq.ParquetFile(quotes_path).metadata.num_rows
trades_rows = pq.ParquetFile(trades_path).metadata.num_rows

print(f"Quotes rows: {quotes_rows:,}")
print(f"Trades rows: {trades_rows:,}")


Quotes rows: 1,957,290,808
Trades rows: 89,518,138


In [6]:
import pyarrow.parquet as pq
import os

DATA_DIR = "~/Documents/TAQData"
quotes_path = os.path.expanduser(f"{DATA_DIR}/data_quotes_2023_12_15.parquet")
trades_path = os.path.expanduser(f"{DATA_DIR}/data_trades_2023_12_15.parquet")

def parquet_date_range(path, date_col):
    pf = pq.ParquetFile(path)
    for rg in range(pf.num_row_groups):
        df = pf.read_row_group(rg, columns=[date_col]).to_pandas()
        if not df.empty:
            dmin, dmax = df[date_col].min(), df[date_col].max()
            return dmin, dmax
    return None, None

quote_min, quote_max = parquet_date_range(quotes_path, "DATE")
trade_min, trade_max = parquet_date_range(trades_path, "date")

print(f"Quotes date range: {quote_min} → {quote_max}")
print(f"Trades date range: {trade_min} → {trade_max}")


Quotes date range: 2023-12-15 → 2023-12-15
Trades date range: 2023-12-15 → 2023-12-15


In [7]:
# 1) What symbols are actually present? (first ~20 row groups, TRADES)
import pyarrow.parquet as pq, pandas as pd, os
TRADES_FILE = os.path.expanduser("~/Documents/TAQData/data_trades_2023_12_15.parquet")
pf = pq.ParquetFile(TRADES_FILE)

seen = []
for rg in range(min(20, pf.num_row_groups)):
    df = pf.read_row_group(rg, columns=["sym_root","sym_suffix"]).to_pandas()
    df["sym_root"]  = df["sym_root"].astype(str).str.strip().str.upper()
    df["sym_suffix"]= df["sym_suffix"].astype(str).fillna("").str.strip().str.upper()
    seen.append(df.groupby(["sym_root","sym_suffix"]).size().reset_index(name="cnt"))
present = pd.concat(seen).groupby(["sym_root","sym_suffix"])["cnt"].sum().reset_index().sort_values("cnt", ascending=False)
print(present.head(25))


    sym_root sym_suffix     cnt
24      AAPL       NONE  625486
122     ADBE       NONE  140747
1         AA       NONE  128766
15       AAL       NONE   94629
52       ABT       NONE   77437
45      ABNB       NONE   76272
133      ADI       NONE   73992
32      ABBV       NONE   69560
165      AEE       NONE   68983
136      ADM       NONE   50396
91       ACN       NONE   48587
0          A       NONE   46061
75      ACGL       NONE   45061
150     ADSK       NONE   43176
180      AEO       NONE   42705
144      ADP       NONE   38899
176      AEM       NONE   35980
47       ABR       NONE   35705
19      AAOI       NONE   35294
10      AADI       NONE   32126
183      AEP       NONE   30936
21       AAP       NONE   28638
61      ACAD       NONE   26610
79      ACHR       NONE   26189
82       ACI       NONE   24420


In [8]:
#please run polar_try_partitioned.py for quotes and sepertely for trades by location
import polars as pl
pl.Config.set_tbl_cols(50) 
pl.Config.set_tbl_width_chars(200)
pl.Config.set_tbl_rows(200) #polars.Config.tbl_rows = 50
#please run rename.py shall rename the cols of trades to uppercase and save since easier for joining

polars.config.Config

In [369]:
import polars as pl
import os

# Define the directory and pattern
directory = "/home/amazon/Documents/TAQData/processed_output_trades_upper"
pattern = os.path.join(directory, "chunk_*.parquet")

# Create a LazyFrame from all matching Parquet files
lazy_df = pl.scan_parquet(pattern)

# Access the schema (column names and types)
schema = lazy_df.collect_schema()

# Print the schema
print(schema)

Schema({'DATE': Date, 'TIME_M': Time, 'TIME_M_NANO': Int16, 'EX': String, 'SYM_ROOT': String, 'SYM_SUFFIX': String, 'TR_SCOND': String, 'SIZE': Int32, 'PRICE': Float64, 'TR_STOP_IND': String, 'TR_CORR': String, 'TR_SEQNUM': Int64, 'TR_ID': String, 'TR_SOURCE': String, 'TR_RF': String, 'PART_TIME': Time, 'PART_TIME_NANO': Int16, 'TRF_TIME': Time, 'TRF_TIME_NANO': Int16, 'TTE_IND': String})


In [371]:
directory = Path("/home/amazon/Documents/TAQData/processed_output_trades_upper/")
pattern = os.path.join(directory, "chunk_*.parquet")
# Lazy load merged dataset
lf = pl.scan_parquet(pattern)

# Filter to ticker
base = (
    lf.filter(pl.col("SYM_ROOT") == 'UBER')
      .sort("TR_SEQNUM")
)

print(f"Null TR_SEQNUM : {  base.filter(pl.col("TR_SEQNUM").is_null()).count().collect().item(0, 0) } ")

Null TR_SEQNUM : 0 


In [14]:
print(lazy_df.filter(pl.col("SYM_ROOT") == "AAPL").head(10).collect())

shape: (10, 14)
┌────────────┬──────────────────┬─────┬────────┬───┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M           ┆ EX  ┆ BID    ┆ … ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---              ┆ --- ┆ ---    ┆   ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ time             ┆ str ┆ f64    ┆   ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪══════════════════╪═════╪════════╪═══╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-12-15 ┆ 03:59:00.1149253 ┆ Z   ┆ 0.0    ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 77               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-12-15 ┆ 03:59:00.1854122 ┆ K   ┆ 0.0    ┆ … ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│            ┆ 94               ┆     ┆        ┆   ┆           ┆           ┆          ┆            │
│ 2023-12-15 ┆ 04:00:00.0044523 ┆ K   ┆ 197.53 ┆ … ┆ null      ┆ N         

In [7]:
import polars as pl
import os

# Define the directory and pattern
directory = "../TAQData/processed_output_quotes"
pattern = os.path.join(directory, "chunk_*.parquet")

# Create a LazyFrame from all matching Parquet files
lazy_df = pl.scan_parquet(pattern)

# Access the schema (column names and types)
schema = lazy_df.collect_schema()

# Print the schema
print(schema)

Schema({'DATE': Date, 'TIME_M': Time, 'EX': String, 'BID': Float64, 'BIDSIZ': Int64, 'ASK': Float64, 'ASKSIZ': Int64, 'QU_COND': String, 'QU_SEQNUM': Int64, 'NATBBO_IND': String, 'QU_CANCEL': String, 'QU_SOURCE': String, 'SYM_ROOT': String, 'SYM_SUFFIX': String})


In [8]:
print(lazy_df.filter((pl.col('SYM_ROOT') == 'AAPL') & ( pl.col('BIDSIZ').is_not_null() )).head(10).collect() )

shape: (10, 14)
┌────────────┬────────────────────┬─────┬────────┬────────┬────────┬────────┬─────────┬───────────┬────────────┬───────────┬───────────┬──────────┬────────────┐
│ DATE       ┆ TIME_M             ┆ EX  ┆ BID    ┆ BIDSIZ ┆ ASK    ┆ ASKSIZ ┆ QU_COND ┆ QU_SEQNUM ┆ NATBBO_IND ┆ QU_CANCEL ┆ QU_SOURCE ┆ SYM_ROOT ┆ SYM_SUFFIX │
│ ---        ┆ ---                ┆ --- ┆ ---    ┆ ---    ┆ ---    ┆ ---    ┆ ---     ┆ ---       ┆ ---        ┆ ---       ┆ ---       ┆ ---      ┆ ---        │
│ date       ┆ time               ┆ str ┆ f64    ┆ i64    ┆ f64    ┆ i64    ┆ str     ┆ i64       ┆ str        ┆ str       ┆ str       ┆ str      ┆ str        │
╞════════════╪════════════════════╪═════╪════════╪════════╪════════╪════════╪═════════╪═══════════╪════════════╪═══════════╪═══════════╪══════════╪════════════╡
│ 2023-12-15 ┆ 03:59:00.114925377 ┆ Z   ┆ 0.0    ┆ 0      ┆ 0.0    ┆ 0      ┆ L       ┆ 1103      ┆ 1          ┆ null      ┆ N         ┆ AAPL     ┆ null       │
│ 2023-12-15 ┆ 03:

In [1]:
#please run query.py for joint dataset
#please run stats.py for top 50

In [29]:
#See the output of stats
import polars as pl

# Define the paths to your files
quotes_file = "stats_out_simple/top50_quotes_by_rows.csv"
trades_file = "stats_out_simple/top50_trades_by_volume.csv"

# Load both files lazily
lazy_quotes_df = pl.scan_csv(quotes_file)
lazy_trades_df = pl.scan_csv(trades_file)

# 1. Sum QUOTE_ROWS from the quotes file
quote_rows_sum = (
    lazy_quotes_df
    .select(pl.col("QUOTES_ROWS").sum().alias("TOTAL_QUOTE_ROWS"))
    .collect()
)

# 2. Sum TRADE_ROWS from the trades file
trade_rows_sum = (
    lazy_trades_df
    .select(pl.col("TRADES_ROWS").sum().alias("TOTAL_TRADE_ROWS"))
    .collect()
)

# Optional: Print the results
print("--- Quote Rows Sum ---")
print(quote_rows_sum)
print("\n--- Trade Rows Sum ---")
print(trade_rows_sum)

--- Quote Rows Sum ---
shape: (1, 1)
┌──────────────────┐
│ TOTAL_QUOTE_ROWS │
│ ---              │
│ i64              │
╞══════════════════╡
│ 328673090        │
└──────────────────┘

--- Trade Rows Sum ---
shape: (1, 1)
┌──────────────────┐
│ TOTAL_TRADE_ROWS │
│ ---              │
│ i64              │
╞══════════════════╡
│ 9324631          │
└──────────────────┘


In [25]:
TRADES_OUT_CSV = OUT_DIR / "top50_trades_by_volume.csv"
QUOTES_OUT_CSV = OUT_DIR / "top50_quotes_by_rows.csv"

print(f"Reading {TRADES_OUT_CSV} and {QUOTES_OUT_CSV} …")
trades_df = pl.read_csv(TRADES_OUT_CSV)
quotes_df = pl.read_csv(QUOTES_OUT_CSV)

print(trades_df.sort("TRADES_ROWS", descending=True))
print(quotes_df.sort("QUOTES_ROWS", descending=True))

Reading stats_out_simple/top50_trades_by_volume.csv and stats_out_simple/top50_quotes_by_rows.csv …
shape: (50, 4)
┌──────────┬────────────┬─────────────┬────────────┐
│ SYM_ROOT ┆ SYM_SUFFIX ┆ TRADES_ROWS ┆ TRADES_VOL │
│ ---      ┆ ---        ┆ ---         ┆ ---        │
│ str      ┆ str        ┆ i64         ┆ i64        │
╞══════════╪════════════╪═════════════╪════════════╡
│ TSLA     ┆            ┆ 1185389     ┆ 159068358  │
│ SPY      ┆            ┆ 669221      ┆ 159259628  │
│ AAPL     ┆            ┆ 625486      ┆ 183784854  │
│ AMZN     ┆            ┆ 492794      ┆ 156092695  │
│ UBER     ┆            ┆ 468538      ┆ 1145449108 │
│ MSFT     ┆            ┆ 447207      ┆ 111464305  │
│ CCCC     ┆            ┆ 339905      ┆ 85960043   │
│ INTC     ┆            ┆ 337153      ┆ 114212069  │
│ PFE      ┆            ┆ 301662      ┆ 287715357  │
│ PLTR     ┆            ┆ 278350      ┆ 130424480  │
│ GOOG     ┆            ┆ 266240      ┆ 87741515   │
│ BAC      ┆            ┆ 242610     

In [23]:
#please run join_trades_quotes.py — combine stats_simple outputs
import polars as pl
from pathlib import Path

OUT_DIR = Path("./stats_out_simple")
TRADES_OUT_CSV = OUT_DIR / "top50_trades_by_volume.csv"
QUOTES_OUT_CSV = OUT_DIR / "top50_quotes_by_rows.csv"
MERGED_OUT_CSV = OUT_DIR / "merged_top50_sorted.csv"

print(f"Reading {TRADES_OUT_CSV} and {QUOTES_OUT_CSV} …")
trades_df = pl.read_csv(TRADES_OUT_CSV)
quotes_df = pl.read_csv(QUOTES_OUT_CSV)

# Join and sort descending by TRADES_VOL
merged = (
    trades_df
    .join(quotes_df, on=["SYM_ROOT", "SYM_SUFFIX"], how="outer")
#    .with_columns([
#        pl.col("TRADES_VOL").fill_null(0),
#        pl.col("TRADES_ROWS").fill_null(0),
#        pl.col("QUOTES_ROWS").fill_null(0)
#    ])
    .filter(pl.col("QUOTES_ROWS").is_not_null())
    .filter(pl.col("TRADES_VOL").is_not_null())
    .sort("TRADES_ROWS", descending=True)
    
)

print("\nMerged:")
print(merged)

merged.write_csv(MERGED_OUT_CSV)
print(f"\n✅ Combined file saved to → {MERGED_OUT_CSV}")


Reading stats_out_simple/top50_trades_by_volume.csv and stats_out_simple/top50_quotes_by_rows.csv …

Merged:
shape: (11, 7)
┌──────────┬────────────┬─────────────┬────────────┬────────────────┬──────────────────┬─────────────┐
│ SYM_ROOT ┆ SYM_SUFFIX ┆ TRADES_ROWS ┆ TRADES_VOL ┆ SYM_ROOT_right ┆ SYM_SUFFIX_right ┆ QUOTES_ROWS │
│ ---      ┆ ---        ┆ ---         ┆ ---        ┆ ---            ┆ ---              ┆ ---         │
│ str      ┆ str        ┆ i64         ┆ i64        ┆ str            ┆ str              ┆ i64         │
╞══════════╪════════════╪═════════════╪════════════╪════════════════╪══════════════════╪═════════════╡
│ TSLA     ┆            ┆ 1185389     ┆ 159068358  ┆ TSLA           ┆                  ┆ 8847235     │
│ SPY      ┆            ┆ 669221      ┆ 159259628  ┆ SPY            ┆                  ┆ 22131374    │
│ AAPL     ┆            ┆ 625486      ┆ 183784854  ┆ AAPL           ┆                  ┆ 9129733     │
│ AMZN     ┆            ┆ 492794      ┆ 156092695  ┆

/tmp/ipykernel_1305903/1662951033.py:17: DeprecationWarning: use of `how='outer'` should be replaced with `how='full'`.
(Deprecated in version 0.20.29)
  .join(quotes_df, on=["SYM_ROOT", "SYM_SUFFIX"], how="outer")


In [34]:
#please run top50_trades_corresponding_quotes.py
'''
   • TRADES saved rows total: 9,324,631
   • QUOTES saved rows total: 128,520,108
   • Summary CSV: /home/amazon/gitcode/WithAmitG/TAQCode/stats_out_simple/merged_top50_sorted.csv
'''
'''
OUT_DIR = Path("./stats_out_simple")
MERGED_OUT_CSV = OUT_DIR / "merged_top50_sorted.csv"
print(f"Reading {MERGED_OUT_CSV} …")
merged_df = pl.read_csv(MERGED_OUT_CSV)
print(merged_df)
'''

'\nOUT_DIR = Path("./stats_out_simple")\nMERGED_OUT_CSV = OUT_DIR / "merged_top50_sorted.csv"\nprint(f"Reading {MERGED_OUT_CSV} …")\nmerged_df = pl.read_csv(MERGED_OUT_CSV)\nprint(merged_df)\n'

In [211]:
#please run top50_trades_corresponding_quotes.py
#please run top50_trades_corresponding_quotes_persist.py
!du -hc ~/Documents/TAQData/merged_output_top50/merged_all.parquet
print('shows top 50 tickers TAQ is not big')

367M	/home/amazon/Documents/TAQData/merged_output_top50/merged_all.parquet
367M	total
shows top 50 tickers TAQ is not big


In [373]:
directory = Path("/home/amazon/Documents/TAQData/processed_output_trades_upper_top50/")
pattern = os.path.join(directory, "*chunk_*.parquet")
# Lazy load merged dataset
lf = pl.scan_parquet(pattern)

# Filter to ticker
base = (
    lf.filter(pl.col("SYM_ROOT") == 'UBER')
      .sort("TR_SEQNUM")
)

print(f"Null TR_SEQNUM : {  base.filter(pl.col("TR_SEQNUM").is_null()).count().collect().item(0, 0) } ")

Null TR_SEQNUM : 0 


In [374]:
MERGED_FILE = Path("/home/amazon/Documents/TAQData/merged_output_top50/merged_all.parquet")
# Lazy load merged dataset
lf = pl.scan_parquet(str(MERGED_FILE))

# Filter to ticker
base = (
    lf.filter(pl.col("SYM_ROOT") == 'UBER')
      .sort("TR_SEQNUM")
)

print(f"Null TR_SEQNUM : {  base.filter(pl.col("TR_SEQNUM").is_null()).count().collect().item(0, 0) } ")

Null TR_SEQNUM : 63805 


In [404]:
#ms.py for microstructure
%run ms.py

Null TR_SEQNUM : 0 
Null QU_SEQNUM : 424846 
Zero BID and ASK : 20114

🔎 Running microstructure analysis for: UBER
shape: (1, 19)
┌────────┬────────────┬───────────┬─────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────────┬────────┬────────────┐
│ N_ROWS ┆ TOTAL_SIZE ┆ VWAP      ┆ AVG_SPREAD_ ┆ AVG_DEPTH_ ┆ IS_MEAN_BP ┆ IS_MEDIAN_ ┆ IS_MIN_BPS ┆ IS_MAX_BPS ┆ MI_MEAN_BP ┆ MI_MEDIAN_ ┆ MI_MIN_BPS ┆ MI_MAX_BPS ┆ MR_MEAN_BP ┆ MR_MEDIAN_ ┆ MR_MIN_BPS ┆ MR_MAX_BPS ┆ TICKER ┆ FWD_TRADES │
│ ---    ┆ ---        ┆ ---       ┆ BPS         ┆ TOB        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ S          ┆ BPS        ┆ ---        ┆ ---        ┆ ---    ┆ _FOR_MR    │
│ u32    ┆ i32        ┆ f64       ┆ ---         ┆ ---        ┆ ---        ┆ ---        ┆ f64        ┆ f64        ┆ ---        ┆ ---        ┆ f64   

In [405]:
summary_df

N_ROWS,TOTAL_SIZE,VWAP,AVG_SPREAD_BPS,AVG_DEPTH_TOB,IS_MEAN_BPS,IS_MEDIAN_BPS,IS_MIN_BPS,IS_MAX_BPS,MI_MEAN_BPS,MI_MEDIAN_BPS,MI_MIN_BPS,MI_MAX_BPS,MR_MEAN_BPS,MR_MEDIAN_BPS,MR_MIN_BPS,MR_MAX_BPS,TICKER,FWD_TRADES_FOR_MR
u32,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,i32
464114,467555920,61.981882,1559.312367,14.826448,9.146267,0.792581,0.0,568.612585,0.0,0.0,0.0,0.0,-7.500319,-1.1514e-12,-764.720877,537.094589,"""UBER""",10


In [385]:
metrics_lf.head(1).collect()

DATE,TIME_M,TIME_M_NANO,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_STOP_IND,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF,PART_TIME,PART_TIME_NANO,TRF_TIME,TRF_TIME_NANO,TTE_IND,TS,DATE_right,TIME_M_right,EX_right,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,MID,SPREAD,DEPTH_TOB,SIDE,ARR_MID,IS_BPS,MI_BPS,MID_FWD,MR_BPS
date,time,i16,str,str,str,str,i32,f64,str,str,i64,str,str,str,time,i16,time,i16,str,datetime[ns],date,time,str,f64,i64,f64,i64,str,i64,str,str,str,f64,f64,i64,i32,f64,f64,f64,f64,f64
2023-12-15,04:00:01.106477,56,"""P""","""UBER""","""""","""TI""",5,62.53,"""N""","""00""",3875,"""52983525027915""","""C""",null,04:00:01.106449,184,null,0,"""0""",2023-12-15 04:00:01.106477056,2023-12-15,04:00:01.078472448,"""T""",62.3,1,63.02,1,"""R""",1837,"""U""",null,"""C""",62.66,0.72,2,-1,62.66,20.746888,-0.0,62.525,-21.544845


In [412]:
#IS debug
metrics_lf.filter(pl.col('IS_BPS') >= 568 ).select(pl.len()).collect().item() #seeing how many rows have this max IS 

2

In [407]:
#IS debug
metrics_lf.filter(pl.col('IS_BPS') >= 568 ).head(3).collect()

DATE,TIME_M,TIME_M_NANO,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_STOP_IND,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF,PART_TIME,PART_TIME_NANO,TRF_TIME,TRF_TIME_NANO,TTE_IND,TS,DATE_right,TIME_M_right,EX_right,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,MID,SPREAD,DEPTH_TOB,SIDE,ARR_MID,IS_BPS,MI_BPS,MID_FWD,MR_BPS
date,time,i16,str,str,str,str,i32,f64,str,str,i64,str,str,str,time,i16,time,i16,str,datetime[ns],date,time,str,f64,i64,f64,i64,str,i64,str,str,str,f64,f64,i64,i32,f64,f64,f64,f64,f64
2023-12-15,10:21:23.029277,696,"""D""","""UBER""","""""",null,100,62.73,"""N""","""00""",620633,"""71679939725906""","""C""","""T""",10:21:23.008376,0,10:21:23.028920,767,"""0""",2023-12-15 10:21:23.029277696,2023-12-15,10:21:23.028263168,"""V""",56.0,1,62.71,5,"""R""",13984555,"""A""",null,"""C""",59.355,6.71,6,1,59.355,568.612585,0.0,62.695,-562.715862
2023-12-15,10:22:05.116808,704,"""D""","""UBER""","""""",null,100,62.74,"""N""","""00""",628108,"""71679955518265""","""C""","""T""",10:22:04.973107,0,10:22:05.116448,279,"""0""",2023-12-15 10:22:05.116808704,2023-12-15,10:22:05.112665856,"""V""",56.0,1,62.73,2,"""R""",14144415,"""A""",null,"""C""",59.365,6.73,3,1,59.365,568.516803,0.0,62.725,-565.990061


In [408]:
#IS debug
MERGED_FILE = Path("/home/amazon/Documents/TAQData/merged_output_top50/merged_all.parquet")
OUT_DIR = Path("./ms_out")
OUT_DIR.mkdir(exist_ok=True)

pl.Config.set_tbl_cols(120)
pl.Config.set_tbl_width_chars(240)

# Event-time lead for MR (number of trades ahead)
K_FWD_TRADES = 10

lf = pl.scan_parquet(str(MERGED_FILE))

# Base slice for this ticker with required columns and event-time ordering
base = (
    lf.filter(pl.col("SYM_ROOT") == 'UBER').sort(["TS","QU_SEQNUM"])
)
traded = base.filter(pl.col("PRICE").is_not_null() & pl.col("SIZE").is_not_null())
import polars as pl

date_str = "2023-12-15 16:06:35.182679552"

traded.filter(
    pl.col("TS") <= pl.lit(date_str).str.strptime(
        pl.Datetime("ns"), 
        format="%Y-%m-%d %H:%M:%S%.f"  # <-- The fix is here
    )
).sort('TIME_M', descending=True).select(['PRICE','QU_SEQNUM']).head(10).collect(engine='streaming')

PRICE,QU_SEQNUM
f64,i64
61.7,68858735
61.7,68858735
61.7,68858727
61.7,68858727
61.69,68858726
61.7,68858723
61.7,68858723
61.7,68858694
61.7,68858683


In [414]:
#IS debug
metrics_lf.sort('IS_BPS', descending=True ).select(['SIDE','PRICE','ARR_MID', 'IS_BPS', 'QU_SEQNUM', 'TR_SEQNUM']).head(10).collect(engine='streaming')

SIDE,PRICE,ARR_MID,IS_BPS,QU_SEQNUM,TR_SEQNUM
i32,f64,f64,f64,i64,i64
1,62.73,59.355,568.612585,13984555,620633
1,62.74,59.365,568.516803,14144415,628108
1,62.745,59.375,567.578947,13940317,618884
1,62.745,59.375,567.578947,14346462,636591
1,62.7501,59.38,567.547996,14334244,635934
1,62.75,59.38,567.531155,13919136,617938
1,62.75,59.38,567.531155,13921009,618037
1,62.75,59.38,567.531155,13921009,618038
1,62.75,59.38,567.531155,13919136,617937


In [413]:
#IS debug
target_qu_seqnum= 13984555
MERGED_FILE = Path("/home/amazon/Documents/TAQData/merged_output_top50/merged_all.parquet")
base = (
    lf.filter(pl.col("SYM_ROOT") == 'UBER').sort(["TS","QU_SEQNUM"])
)
traded = base.filter(pl.col("PRICE").is_not_null() & pl.col("SIZE").is_not_null())
traded.filter(
   # pl.col("QU_SEQNUM").is_in ( [x + y for x, y in zip( [target_qu_seqnum] * len(range (-50,51)  ) , [a for a in range (-50,51) ] ) ] )
    pl.col("QU_SEQNUM").is_in ( [1489851  ] )
).sort('QU_SEQNUM', descending=True).head(10).collect(engine='streaming')

DATE,TIME_M,TIME_M_NANO,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_STOP_IND,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF,PART_TIME,PART_TIME_NANO,TRF_TIME,TRF_TIME_NANO,TTE_IND,TS,DATE_right,TIME_M_right,EX_right,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,MID,SPREAD,DEPTH_TOB
date,time,i16,str,str,str,str,i32,f64,str,str,i64,str,str,str,time,i16,time,i16,str,datetime[ns],date,time,str,f64,i64,f64,i64,str,i64,str,str,str,f64,f64,i64
2023-12-15,09:33:54.511874,48,"""D""","""UBER""","""""","""4 I""",50,61.5586,"""N""","""00""",91136,"""71675342769208""","""C""","""T""",09:33:54.511142,995,09:33:54.511519,317,"""1""",2023-12-15 09:33:54.511874048,2023-12-15,09:33:54.511869696,"""N""",61.51,1,61.6,2,"""R""",1489851,"""A""",null,"""C""",61.555,0.09,3
2023-12-15,09:33:54.511871,232,"""N""","""UBER""","""""","""F""",246,61.6,"""N""","""00""",91135,"""52983525225281""","""C""",null,09:33:54.511843,890,null,0,"""1""",2023-12-15 09:33:54.511871232,2023-12-15,09:33:54.511869696,"""N""",61.51,1,61.6,2,"""R""",1489851,"""A""",null,"""C""",61.555,0.09,3


In [329]:
#IS_BPS_MAX based on diff between arrival and price

In [417]:
#MR debug
# metrics_lf.filter(pl.col('MR_BPS') >= 10000 ).collect(engine='streaming')
metrics_lf.sort('MR_BPS', descending=True ).select(['SIDE','MID_FWD','MID', 'MR_BPS', 'QU_SEQNUM']).head(10).collect(engine='streaming')

SIDE,MID_FWD,MID,MR_BPS,QU_SEQNUM
i32,f64,f64,f64,i64
1,59.375,62.745,537.094589,13921145
1,59.38,62.75,537.051793,13919068
1,59.38,62.75,537.051793,13917599
1,59.38,62.75,537.051793,13921007
1,59.38,62.745,536.297713,13920997
1,59.38,62.745,536.297713,13921003
1,59.355,62.715,535.756996,12936217
1,59.36,62.72,535.714286,13984223
1,59.36,62.72,535.714286,13984223


In [418]:
target_qu_seqnum= 13921145
MERGED_FILE = Path("/home/amazon/Documents/TAQData/merged_output_top50/merged_all.parquet")
base = (
    lf.filter(pl.col("SYM_ROOT") == 'UBER').sort(["TS","QU_SEQNUM"])
)
traded = base.filter(pl.col("PRICE").is_not_null() & pl.col("SIZE").is_not_null())
traded.filter(
   pl.col("QU_SEQNUM").is_in ( [x + y for x, y in zip( [target_qu_seqnum] * len(range (-50,51)  ) , [a for a in range (-50,51) ] ) ] )
    # pl.col("QU_SEQNUM").is_in ( [1489851  ] )
).sort('QU_SEQNUM', descending=True).head(10).collect(engine='streaming')

DATE,TIME_M,TIME_M_NANO,EX,SYM_ROOT,SYM_SUFFIX,TR_SCOND,SIZE,PRICE,TR_STOP_IND,TR_CORR,TR_SEQNUM,TR_ID,TR_SOURCE,TR_RF,PART_TIME,PART_TIME_NANO,TRF_TIME,TRF_TIME_NANO,TTE_IND,TS,DATE_right,TIME_M_right,EX_right,BID,BIDSIZ,ASK,ASKSIZ,QU_COND,QU_SEQNUM,NATBBO_IND,QU_CANCEL,QU_SOURCE,MID,SPREAD,DEPTH_TOB
date,time,i16,str,str,str,str,i32,f64,str,str,i64,str,str,str,time,i16,time,i16,str,datetime[ns],date,time,str,f64,i64,f64,i64,str,i64,str,str,str,f64,f64,i64
2023-12-15,10:21:06.493738,496,"""P""","""UBER""","""""",null,100,62.745,"""N""","""00""",618062,"""52983541938498""","""C""",null,10:21:06.493711,339,null,0,"""0""",2023-12-15 10:21:06.493738496,2023-12-15,10:21:06.493734912,"""V""",62.74,1,62.76,10,"""R""",13921171,"""A""",null,"""C""",62.75,0.02,11
2023-12-15,10:21:06.493537,24,"""T""","""UBER""","""""","""F I""",7,62.75,"""N""","""00""",618061,"""62879398644560""","""C""",null,10:21:06.493182,262,null,0,"""1""",2023-12-15 10:21:06.493537024,2023-12-15,10:21:06.493534720,"""Z""",62.74,3,62.75,2,"""R""",13921145,"""A""",null,"""C""",62.745,0.01,5
2023-12-15,10:21:06.493533,952,"""T""","""UBER""","""""","""F I""",5,62.75,"""N""","""00""",618059,"""62879398644558""","""C""",null,10:21:06.493182,262,null,0,"""1""",2023-12-15 10:21:06.493533952,2023-12-15,10:21:06.493529600,"""P""",62.74,6,62.75,3,"""R""",13921142,"""A""",null,"""C""",62.745,0.01,9
2023-12-15,10:21:06.493533,952,"""T""","""UBER""","""""","""F""",100,62.75,"""N""","""00""",618060,"""62879398644559""","""C""",null,10:21:06.493182,262,null,0,"""1""",2023-12-15 10:21:06.493533952,2023-12-15,10:21:06.493529600,"""P""",62.74,6,62.75,3,"""R""",13921142,"""A""",null,"""C""",62.745,0.01,9
2023-12-15,10:21:06.493498,112,"""K""","""UBER""","""""","""I""",1,62.75,"""N""","""00""",618058,"""52983525291570""","""C""",null,10:21:06.493267,0,null,0,"""0""",2023-12-15 10:21:06.493498112,2023-12-15,10:21:06.493489408,"""M""",62.69,3,62.8,2,"""R""",13921134,"""A""",null,"""C""",62.745,0.11,5
2023-12-15,10:21:06.493404,672,"""T""","""UBER""","""""","""F""",100,62.75,"""N""","""00""",618056,"""62879398644556""","""C""",null,10:21:06.493043,695,null,0,"""1""",2023-12-15 10:21:06.493404672,2023-12-15,10:21:06.493404672,"""P""",62.74,4,62.75,3,"""R""",13921114,"""U""",null,"""C""",62.745,0.01,7
2023-12-15,10:21:06.493404,672,"""T""","""UBER""","""""","""F I""",95,62.75,"""N""","""00""",618057,"""62879398644557""","""C""",null,10:21:06.493043,695,null,0,"""1""",2023-12-15 10:21:06.493404672,2023-12-15,10:21:06.493404672,"""P""",62.74,4,62.75,3,"""R""",13921114,"""U""",null,"""C""",62.745,0.01,7
2023-12-15,10:21:06.493401,600,"""T""","""UBER""","""""","""F""",100,62.75,"""N""","""00""",618055,"""62879398644555""","""C""",null,10:21:06.493043,695,null,0,"""1""",2023-12-15 10:21:06.493401600,2023-12-15,10:21:06.493399296,"""N""",62.74,6,62.75,2,"""R""",13921112,"""A""",null,"""C""",62.745,0.01,8
2023-12-15,10:21:06.493395,712,"""T""","""UBER""","""""","""F""",100,62.745,"""N""","""00""",618053,"""62879398644553""","""C""",null,10:21:06.493043,695,null,0,"""1""",2023-12-15 10:21:06.493395712,2023-12-15,10:21:06.493394176,"""T""",62.74,2,62.75,3,"""R""",13921108,"""U""",null,"""C""",62.745,0.01,5


In [398]:
#MR_BPS_MAX >= 537 because huge difference in MID and MID_FWD

In [419]:
#MI debug
# metrics_lf.filter(pl.col('MR_BPS') >= 10000 ).collect(engine='streaming')
metrics_lf.sort('MI_BPS', descending=True ).select(['SIDE','MID','ARR_MID', 'MI_BPS', 'QU_SEQNUM', 'TR_SEQNUM']).head(10).collect(engine='streaming')

SIDE,MID,ARR_MID,MI_BPS,QU_SEQNUM,TR_SEQNUM
i32,f64,f64,f64,i64,i64
1,63.08,63.08,0.0,38220100,1778120
1,63.08,63.08,0.0,38219439,1777888
-1,63.085,63.085,-0.0,38218794,1777878
1,63.08,63.08,0.0,38218785,1777877
1,63.08,63.08,0.0,38218758,1777875
1,63.07,63.07,0.0,38218703,1777874
-1,63.085,63.085,-0.0,38218570,1777870
1,63.08,63.08,0.0,38218561,1777869
1,63.08,63.08,0.0,38218561,1777868


In [ ]:
#MI_BPS_MAX is 0 because arrival_mid is very close to mid price